
# Diseño de una propuesta de optimización del sistema de alertas tempranas por deforestación para Colombia, caso de estudio “Corazón de la Amazonia”

## Descarga de imagenes 

### Cristhian Forero Castro

In [1]:
#Descomente la siguiente línea para instalar [geemap](https://geemap.org) si es necesario.
#!pip install geemap
#geemap.update_package()

In [2]:
import ee
import geemap
import os
#import ipyleaflet
#import ipywidgets as widgets

In [5]:
#Map = geemap.Map()
#Map

In [4]:
vis_params = {'color': '000000ff', 'width': 2, 'lineType': 'solid', 'fillColor': '000000a8'}
Map = geemap.Map(center=[4, -74], zoom=6)
areaestudio = 'F:/Cristhian/Tesis_capitulo_2/Imagenes_areasdeanalisis_marginaldelaselva/areaestudio_shp.shp'
#Map.add_shapefile(areaestudio, layer_name="marginal")
#Map
areaestudio = geemap.shp_to_ee(areaestudio)
Map.addLayer(areaestudio, {}, 'Countries')
Map

Map(center=[4, -74], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=Fa…

## Visualización de mosaico

In [6]:
image = ee.Image('LE7_TOA_5YEAR/1999_2003')
landsat_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'gamma': 1.4
}
Map.addLayer(image, landsat_vis, "LE7_TOA_5YEAR/1999_2003", True, 0.7)

In [8]:
#dem = ee.Image('USGS/SRTMGL1_003')
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()
#roi= ee.Geometry.Polygon(areaestudio)
#Fechas 
desde='2021-02-01'
hasta='2021-09-30'

---------
# Descarga de imagenes Sentinel 2 SR

In [18]:
# Función para enmascarar nubes usando la banda Sentinel-2 QA.
def maskS2clouds(image):
    qa = image.select('QA60')

  # Los bits 10 y 11 son nubes y cirrus.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
  # Ambas mascaras deben establecerse en cero.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Devuelve los datos enmascarados
    return image.updateMask(mask).divide(1) \
        .select('B4', 'B8', 'B11', 'B12').reproject('EPSG:32618', None, 3).clip(roi) \
        .copyProperties(image, ["system:time_start"])


In [19]:
# Se calcula la mediana de los datos.
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi)\
    .filterDate(desde,hasta) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
    .map(maskS2clouds)
composite = collection.median().clip(roi)
# Resultados.
Map.addLayer(composite, {'bands': ['B8', 'B11', 'B12'], 'min': 0, 'max': 3000}, 'RGB')

#### Imágenes a exportar

In [20]:
print(collection.aggregate_array('system:index').getInfo())

['20210214T151659_20210214T151702_T18NXH', '20210301T151701_20210301T152218_T18NXH', '20210321T151701_20210321T151703_T18NXH', '20210624T151709_20210624T151704_T18NXH', '20210808T151711_20210808T151707_T18NXH', '20210818T151711_20210818T152108_T18NXH', '20210823T151659_20210823T151702_T18NXH', '20210828T151711_20210828T151706_T18NXH']


#### Descarga de imágenes seleccionadas

In [21]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
geemap.ee_export_image_collection(collection,scale=30, out_dir=out_dir)

Total number of images: 8

Exporting 1/8: 20210214T151659_20210214T151702_T18NXH.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\20210214T151659_20210214T151702_T18NXH.tif


Exporting 2/8: 20210301T151701_20210301T152218_T18NXH.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\20210301T151701_20210301T152218_T18NXH.tif


Exporting 3/8: 20210321T151701_20210321T151703_T18NXH.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\20210321T151701_20210321T151703_T18NXH.tif


Exporting 4/8: 20210624T151709_20210624T151704_T18NXH.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\20210624T151709_20210624T151704_T18NXH.tif


Exporting 5/8: 20210808T151711_20210808T151707_T18NXH.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\20210808T151711_20210808T151707_T18NXH.tif


Exporting 6/8: 20210818T151711_20210818T152108_T18NXH.tif
Gener

----------------------------------------------------------------------------------
# Descarga de imagenes Landsat SR 
#### Cambiar link collection l7 y l8 al igual que las bandas

In [22]:
# Función para máscara de nubes de la banda pixel_qa de datos Landsat 8 SR.

def maskL8sr(image):
  # Bits 3 y 5 nubes y sombras
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
  # Seleccionar la banda pixel qa.
    qa = image.select('pixel_qa')
  # Ambas mascaras deben establecerse en cero.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

    return image.updateMask(mask).divide(1) \
        .select('B4','B5','B6','B7').clip(roi) \
        .copyProperties(image, ["system:time_start"])

collectionlandsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi).filterDate(desde, hasta)\
    .map(maskL8sr)

compositelandsat = collectionlandsat.median().clip(roi)

# Visualización de los resultados
Map.addLayer(compositelandsat, {'bands': ['B4', 'B5', 'B7'], 'min': 0, 'max': 3000}, 'Image')

In [23]:
print(collectionlandsat.aggregate_array('system:index').getInfo())

['LC08_007058_20210214', 'LC08_007058_20210302', 'LC08_007058_20210419', 'LC08_007059_20210622', 'LC08_007059_20210825']


#### Descarga de imágenes seleccionadas

In [24]:
#DESCARGA DE IMÁGENES LANDSAT
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
geemap.ee_export_image_collection(collectionlandsat, out_dir=out_dir, scale=30, region=roi)

Total number of images: 5

Exporting 1/5: LC08_007058_20210214.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\LC08_007058_20210214.tif


Exporting 2/5: LC08_007058_20210302.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\LC08_007058_20210302.tif


Exporting 3/5: LC08_007058_20210419.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\LC08_007058_20210419.tif


Exporting 4/5: LC08_007059_20210622.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\LC08_007059_20210622.tif


Exporting 5/5: LC08_007059_20210825.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\LC08_007059_20210825.tif




-------------------------------------------------------------------------------------------------------------------------------
# Descarga de imagenes Sentinel-1 SAR

In [25]:
# Llamado de la colección de Sentinel-1
def clipSAR(imagesar):
    return imagesar.select('VV').clip(roi) \
        .copyProperties(imagesar, ["system:time_start"])

sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(roi).filterDate(desde, hasta)\
    .map(clipSAR)

# Filtro de la colección.
vh = sentinel1 \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
vv=  sentinel1 \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))

# Seleccionar lo que se desea descargar.
SAR_VH_A = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
SAR_VH_D = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
SAR_VV_A = vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
SAR_VV_D = vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

compositeSAR=SAR_VV_A.median()
# Visualización de la métrica.
Map.addLayer(compositeSAR, {'min': [-25], 'max': [0]}, 'composite')

#### Descarga de imágenes seleccionadas

In [26]:
print(SAR_VV_A.aggregate_array('system:index').getInfo())

['S1B_IW_GRDH_1SDV_20210210T231230_20210210T231255_025550_030B6A_8B4B', 'S1B_IW_GRDH_1SDV_20210222T231230_20210222T231255_025725_03111F_F04D', 'S1B_IW_GRDH_1SDV_20210306T231230_20210306T231255_025900_0316D7_52C2', 'S1B_IW_GRDH_1SDV_20210318T231230_20210318T231255_026075_031C7B_2296', 'S1B_IW_GRDH_1SDV_20210330T231230_20210330T231255_026250_03220A_86BD', 'S1B_IW_GRDH_1SDV_20210411T231230_20210411T231255_026425_032797_97AA', 'S1B_IW_GRDH_1SDV_20210423T231231_20210423T231256_026600_032D36_FABC', 'S1B_IW_GRDH_1SDV_20210505T231231_20210505T231256_026775_0332CE_72A5', 'S1B_IW_GRDH_1SDV_20210517T231232_20210517T231257_026950_033847_A9D4', 'S1B_IW_GRDH_1SDV_20210529T231233_20210529T231258_027125_033D83_9995', 'S1B_IW_GRDH_1SDV_20210610T231234_20210610T231259_027300_0342C3_B840', 'S1B_IW_GRDH_1SDV_20210622T231234_20210622T231259_027475_0347EA_0BC4', 'S1B_IW_GRDH_1SDV_20210704T231235_20210704T231300_027650_034CD6_BC1E', 'S1B_IW_GRDH_1SDV_20210716T231236_20210716T231301_027825_035204_48ED', 'S1B_

In [27]:
#DESCARGA DE IMÁGENES SENTINEL 1 
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#filename = os.path.join(out_dir, 'landsat.tif')
#geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)
geemap.ee_export_image_collection(SAR_VV_A, out_dir=out_dir, scale=30, region=roi)

Total number of images: 19

Exporting 1/19: S1B_IW_GRDH_1SDV_20210210T231230_20210210T231255_025550_030B6A_8B4B.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\S1B_IW_GRDH_1SDV_20210210T231230_20210210T231255_025550_030B6A_8B4B.tif


Exporting 2/19: S1B_IW_GRDH_1SDV_20210222T231230_20210222T231255_025725_03111F_F04D.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\S1B_IW_GRDH_1SDV_20210222T231230_20210222T231255_025725_03111F_F04D.tif


Exporting 3/19: S1B_IW_GRDH_1SDV_20210306T231230_20210306T231255_025900_0316D7_52C2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\S1B_IW_GRDH_1SDV_20210306T231230_20210306T231255_025900_0316D7_52C2.tif


Exporting 4/19: S1B_IW_GRDH_1SDV_20210318T231230_20210318T231255_026075_031C7B_2296.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\crist\Downloads\S1B_IW_GRDH_1SDV_20210318T231230_20210318T231255_026075_031C7B_2296.tif


Exportin

## ...